In [42]:
import sqlite3 as sqlite
import pandas as pd
#Acessando o banco de dados
conn = sqlite.connect('clicksign_test.db')
cursor = conn.cursor()

home_team_goal : pd.DataFrame= pd.read_sql_query("""
        WITH
        base_metric AS (
            SELECT 
                -- Semana começando na segunda
                date(strftime('%s', date) - 86400 * ((strftime('%w', date) + 6) % 7), 'unixepoch') as week,
                home_team_goal
            FROM match
                INNER JOIN team
                    ON team.team_api_id = match.home_team_api_id
        )
        
            
        SELECT 
            week,
            AVG(home_team_goal) OVER (PARTITION BY week) as average_home_team_goals
        FROM base_metric
        GROUP BY week
        ORDER BY week DESC

        
    """, conn)
    
#Finalizando a conexão
conn.commit()
conn.close()
home_team_goal.to_csv('home_team_goal_week_report.csv')